# Problema 1: valutazione di `∇J`

In breve: con il set di istruzioni fornito dall'articolo la proprietà associativa non vale. Bisogna aggiustare le cose.

## Setup

### Algebra degli operatori

Carichiamo il set minimo di dati e istruzioni

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [2]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator

adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪()), X::Matrix) = -X  # 𝒪() - X should be -X
\(ℐ::typeof(ℐ()), A::Matrix) = A    # left division with ℐ() does nothing
*(A::Operator, X::Matrix) = A.op(X)
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

### La rete neurale

Riusiamo le celle di codice già viste...

In [3]:
using OffsetArrays

# funzione di attivazione
h(x)  = tanh(x)
h′(x) = 1 - h(x)^2


# questa funzione, di fatto, effettua il forward pass
function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

Adesso possiamo costruire la rete

In [4]:
## Un po' di parametri che descrivono la rete...

n = [5,4,3,1]   # contiene [n₀...n_N], le dimensioni dei layer
k = 10          # batchsize
N = length(n)-1 # numero di layer (nascosti + output). Dev'essere positivo

init(sizes...) = 0.01randn(sizes...)    # utility function per inizializzare

init (generic function with 1 method)

In [5]:
## Un po' di parametri degni di questo nome

# Creiamo le matrici dei pesi e i vettori dei bias
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here

X₀ = init(n[1],k)         # batch di k pattern (aka i dati)
y  = init(n[end],k);      # y is what we will compare X_N against (aka l'etichetta)

𝓁(x,y)  = sum(abs2,x-y)/2   # loss (errore quadratico)
𝓁′(x,y) = x .- y;           # derivata della loss (w.r.t. output layer)

X, δ = neural_net(Ws_and_bs,X₀) # Inferenza (aka forward pass)
# X e δ hanno i valori per ogni layer (servono per fare il backward pass)

([[-0.010307702276031772 0.0013154599260633815 … 0.012920126517820216 0.020609023467865067; 0.001924231286746853 -0.005245409114810823 … -0.0074707090760647256 0.016808768531731785; … ; 0.006841740319856688 0.01141284796333716 … 0.0008059976520682994 -0.007324034169239899; 0.001984279549567257 0.004305495179837947 … -0.009324269699385442 -0.0004520306558560471], [0.01133527454239448 0.01105392099992945 … 0.011243244023796043 0.011421809222234538; -0.0005348873658132839 -0.00011953989667172488 … -0.0004856416913058117 -0.0006957772800036305; -0.0025293626722278726 -0.0025095091467444374 … -0.002802404512536126 -0.003180086796081113; -0.012380895790132908 -0.012246421039332059 … -0.012610570430455287 -0.01247849587497028], [-0.003200151818909338 -0.003202407932116958 … -0.003204780412595212 -0.0032114497783869538; -0.0008163482986829881 -0.0008167033350213195 … -0.0008119194850029582 -0.0008112193341608541; -0.007566873884330025 -0.007566344330305357 … -0.007578647750904534 -0.0075833406

Adesso possiamo calcolare il gradiente della loss usando la nostra tecnica

In [8]:
## The diagonal matrix
#M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N]) # OLD!
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i]) ∘ ℛ(ones(1,k))] for i=1:N])

#M = Bidiagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N], [ℋ(δ[i]) ∘ ℛ(ones(1,k)) for i=1:(N-1)], :U)
#M = [M [fill(𝒪, N-1); ℋ(δ[N]) ∘ ℛ(ones(1,k))]]


## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## derivata della loss (rispetto all'output layer)
g = [ fill(𝒪(),N-1) ; [𝓁′(X[N],y)] ]      

## Finalmente, il gradiente della loss, usando il backslash
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[8.232623947958088e-9 -1.801903786488388e-8 … 8.076483108525702e-9 -4.413790011747871e-9; 4.783016936184446e-9 -1.0468778901121642e-8 … 4.6923340667404396e-9 -2.564357916605562e-9; 3.03280754455042e-9 -6.638115086945066e-9 … 2.9753622918268387e-9 -1.626031658822852e-9; 5.362360854768757e-8 -1.1736597064307389e-7 … 5.2605871173507496e-8 -2.874954156723778e-8]; [2.0060066723754266e-6; 1.165457878069405e-6; 7.389981921818038e-7; 1.3066114551841698e-5;;];;]
 [[-7.231677258769556e-6 3.7015693483346e-7 1.77437102166741e-6 8.030191710372055e-6; 1.955004499568706e-6 -1.0006786148027616e-7 -4.796817154714158e-7 -2.1708741087732686e-6; 6.7916209687157735e-6 -3.4763243281413215e-7 -1.666398388749739e-6 -7.541544853108877e-6]; [-0.0006365399694799989; 0.00017208158773378434; 0.0005978057442319364;;];;]
 [[-0.00014468614830818053 -3.6692662612222896e-5 -0.00034219231584231384]; [0.04515956640173984;;];;]

## Indagine del problema

Il codice proposto è ben lontano dall'essere un sistema completo che permette di definire un'algebra degli operatori. Piuttosto, i metodi sovraccaricati sono quelli strettamente necessari a far funzionare questo esempio (ossia a presentare ciò che l'articolo voleva presentare, che non è poca cosa).

In altri termini, non possiamo fare tutti i conti che ci pare.

### Valutazione di `∇J`

Abbiamo visto che $\nabla J = M^{\top}\cdot (I-L^{\top})^{-1} \cdot g$

In matematica vale la proprietà associativa, ma è bene tenere a mente che i due prodotti della formula sono ben diversi!
- $M^{\top}\cdot(I-L)^{-\top}$: prodotto tra due matrici di operatori. 
    - Per definirlo, bisogna definire un prodotto riga per colonna in cui il prodotto tra gli ""scalari"" è il prodotto tra omomorfismi, ossia la composizione. 
    - Questo prodotto è più vicino alla composizione, come operazione
- $(I-L^{\top})^{-1}\cdot g$: prodotto tra una matrice di omomorfismi e un vettore di matrici. 
    - Nemmeno questo prodotto viene esplicitamente definito, ma non ce n'è bisogno, perché abbiamo definito il prodotto tra un operatore e una matrice (sarebbe l'operazione interna, nel prodotto che qui stiamo considerando). 
    - Questo prodotto è più vicino alla valutazione, come operazione

In [9]:
print("typeof(g) = $(typeof(g))\n")
for i in 1:length(g)
    print("\t typeof(g[$i]) = $(typeof(g[i]))\n")
end

typeof(g) = Vector{Any}
	 typeof(g[1]) = Operator
	 typeof(g[2]) = Operator
	 typeof(g[3]) = Matrix{Float64}


***OSS***: Nel termine noto `g` c'era `𝒪` nel codice originale, non `𝒪()`. Sono due cose molto diverse! (`𝒪()` è un operatore). Ho fixato io la cosa cambiando la regola di sottrazione

Al netto degli zeri `𝒪`, `g` è un array che ha matrici come elementi.

Quello che fa l'algoritmo:
- Calcola `x = ImL' \ g` risolvendo il sistema tramite sostituzione all'indietro
    - Fa operazioni "semplici": Operatore * matrice, somma tra matrici, divisione a sx. per l'operatore identità
    - Restituisce un `Vector{Matrix}`
- Calcola `M' * x` 
    - Fa operazioni semplici: Operatore * matrice, somma tra matrici
    - Restituisce un `Vector{Matrix}`

In [10]:
print("typeof(ImL) = $(typeof(ImL))\n")
print("size(ImL) = $(size(ImL))\n")
print("eltype(ImL) = $(eltype(ImL))\n")

typeof(ImL) = Bidiagonal{Operator, Vector{Operator}}
size(ImL) = (3, 3)
eltype(ImL) = Operator


#### Il tipo di `ImL` è importante

Non è una dimostrazione, ma diamo una prova del fatto che veramente il codice si basa sul fatto che `ImL'` è triangolare superiore, e che il calcolo `ImL' \ g` è fattibile, con le regole che abbiamo, perché viene fatta la *backsubstitution*.

In [11]:
try
    Matrix(ImL') \ g
catch e
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg) - 1]
end

"MethodError: no method matching one(::Type{Operator})"

Julia tenta di fare la fattorizzazione LU. Parleremo di questo anche nel Notebook `2_Inversione.ipynb`

### (Non) Associatività di `*`

Tutto questo per dire che **non abbiamo la proprietà associativa, nella valutazione di `∇J`, perché i due prodotti sono diversi**

In [14]:
# estraiamo da M solo le derivate parziali rispetto ai pesi
A = Diagonal((x->x[1]).(diag(M)))
display(A)

3×3 Diagonal{Operator, Vector{Operator}}:
 ℋ(4, 10)∘ℛ(5, 10)  ⋅                  ⋅
 ⋅                  ℋ(3, 10)∘ℛ(4, 10)  ⋅
 ⋅                  ⋅                  ℋ(1, 10)∘ℛ(3, 10)

In [15]:
display(ImL)

3×3 Bidiagonal{Operator, Vector{Operator}}:
 I                  ⋅                  ⋅
 -ℋ(3, 10)∘ℒ(3, 4)  I                  ⋅
 ⋅                  -ℋ(1, 10)∘ℒ(1, 3)  I

Il prodotto `ImL * A` ha senso: entrambe sono matrici 3x3, e se ho fatto bene i conti, gli operandi (aka elementi delle matrici) "sensibili" sono componibili

In [16]:
try 
    ImL * A
catch e
    showerror(stdout, e)
end

MethodError: no method matching *(::Operator, ::Operator)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:642
  *(::Operator, ::Matrix)
   @ Main ~/Scrivania/MA-project/Codice/src/Notebooks problemi/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:11
  *(::BitMatrix, ::UniformScaling)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.12.4+0.x64.linux.gnu/share/julia/stdlib/v1.12/LinearAlgebra/src/uniformscaling.jl:264
  ...


#### Aggiustamenti

Si può tirare qualche "martellata" che fa funzionare le cose in qualche caso particolare. Consideriamo una versione più semplice del prodotto:

In [17]:
try
    Diagonal(ImL) * A
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching *(::Operator, ::Operator)"

L'errore dice che il comportamento di `*` tra operatori è indefinito. Ma noi lo sappiamo benissimo cosa vuol dire: vuol dire composizione!

In [18]:
*(A::Operator, B::Operator) = A ∘ B

* (generic function with 191 methods)

In [19]:
try
    Diagonal(ImL) * A
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

3×3 Diagonal{Operator, Vector{Operator}}:
 I∘ℋ(4, 10)∘ℛ(5, 10)  ⋅                    ⋅
 ⋅                    I∘ℋ(3, 10)∘ℛ(4, 10)  ⋅
 ⋅                    ⋅                    I∘ℋ(1, 10)∘ℛ(3, 10)

L'esperienza mi ha mostrato che questa regola può far funzionare diversi casi semplici, ma in numerosi altri casi servono altre regole

***OSS***: Ovviamente, la questione su se il prodotto abbia senso o meno rimane aperta in ogni caso.

#### Non controllo di ammissibilità

**Attenzione!** Come già accennato, potremmo star puntando la pistola contro il nostro piede, e il sistema ci avviserebbe che premendo il grilletto il proiettile parte contro il nostro corpo solo dopo che il mignolino è saltato per aria!

In termini meno poetici, potremmo star componendo omomorfismi che non si possono comporre, e il sistema ce lo lascerebbe fare

In [20]:
# Julia ci lascia inserire il proiettile...
compWrong = ℒ([1 2 3]) * ℒ([2 -1; 1 2])

ℒ(1, 3)∘ℒ(2, 2)

In [21]:
# ... e ci avverte solo dopo lo sparo!
try 
    compWrong * rand(2,3)
catch e
    print("Sparandoti contro il piede, ti farai male al piede!\n\n")
    print("Come dici? Potevo dirtelo prima? Avrei dovuto non lasciarti puntare il fucile?\n")
    print("Beh, e che ne sapevo che avevi intenzione di sparare?!?")
end

Sparandoti contro il piede, ti farai male al piede!

Come dici? Potevo dirtelo prima? Avrei dovuto non lasciarti puntare il fucile?
Beh, e che ne sapevo che avevi intenzione di sparare?!?